## Add channel files based on the EEG with fixes.

This notebook replaces the `channels.tsv` files with those generated by EEGLAB BidsTools
for the Attention Shift data.


In [1]:
import os
import json
import datetime
import pandas as pd
from hed.tools import get_file_list, get_value_dict, HedLogger

# Variables to set for the specific dataset
bids_root_path = '/XXX/AttentionShiftWorkingPhaseTwo'
exclude_dirs = ['sourcedata', 'stimuli', 'code']
log_name = 'attention_shift_15_fix_channels_log'
sampling_rate_file = os.path.realpath(os.path.join(bids_root_path, 'code/samplingRates.tsv'))
channel_file = os.path.realpath(os.path.join(bids_root_path, "code/channelsOriginal.json"))

# Set up the logger
log_file_name = f"code/curation_logs/{log_name}.txt"
logger = HedLogger(name=log_name)

# Load the channels file
with open(channel_file, 'r') as fp:
    channel_info = json.load(fp)

# Construct the lists of the EEG files
eeg_files = get_file_list(bids_root_path, extensions=[".set"], name_suffix="_eeg", exclude_dirs=exclude_dirs)

# Get the sample rate dictionary
sampling_dict = get_value_dict(sampling_rate_file)

chan_dict = {'LM': ('LM', 'MISC', 'microV', 'Left mastoid.'),
             'REye': ('REye', 'EOG', 'microV', 'Right EOG channel.'),
             'LEye': ('LEye', 'EOG', 'microV', 'Right EOG channel.')
             }

for file_name in eeg_files:
    basename = os.path.basename(file_name)
    channels = channel_info[basename]
    logger.add(basename, f"EEG has {len(channels)} channels")

    data = {'name': channels}
    df = pd.DataFrame(data)
    df[['type', 'units', 'description']] = 'n/a'
    names = df['name']
    for index, name in names.iteritems():
        row = chan_dict.get(name, None)
        if not row:
            row = (name, 'EEG', 'microV', 'n/a')
        df.loc[index, 'name'] = row[0]
        df.loc[index, 'type'] = row[1]
        df.loc[index, 'units'] = row[2]
        df.loc[index, 'description'] = row[3]
    EEG = sum(df['type'].map(str) == 'EEG')
    EOG = sum(df['type'].map(str) == 'EOG')
    OTHER = sum(df['type'].map(str) == 'OTHER')
    MISC = sum(df['type'].map(str) == 'MISC')
    logger.add(basename, f"{EEG} EEG channels, {EOG} EOG channels, {OTHER} OTHER channels {MISC} MISC channels")
    file_new = file_name[:-7] + "channels.tsv"
    df.to_csv(file_new, sep='\t', index=False)
    logger.add(basename, f"Saved as {os.path.basename(file_new)}")
    eeg_json_file = file_name[:-4] + ".json"
    with open(eeg_json_file, 'r') as fp:
        eeg_json = json.load(fp)
    logger.add(basename, f"Adjusted the MiscChannelCount to 1")
    eeg_json['MiscChannelCount'] = 1
    with open(eeg_json_file, 'w') as fp:
        json.dump(eeg_json, fp, indent=4)

# Output and save the log
log_string = "\n\nLog output:\n" + logger.get_log_string()
error_string = "\n\nERROR Summary:\n" + logger.get_log_string(level="ERROR")
print(log_string)
print(error_string)

save_path = os.path.join(bids_root_path, log_file_name)
with open(save_path, "w") as fp:
    fp.write(f"{log_file_name} {datetime.datetime.now()}\n")
    fp.write(log_string)
    fp.write(error_string)



Log output:
attention_shift_15_fix_channels_log: Level None
sub-001_task-AuditoryVisualShift_run-01_eeg.set:
	[ EEG has 36 channels]
	[ 33 EEG channels, 2 EOG channels, 0 OTHER channels 1 MISC channels]
	[ Saved as sub-001_task-AuditoryVisualShift_run-01_channels.tsv]
	[ Adjusted the MiscChannelCount to 1]
sub-002_task-AuditoryVisualShift_run-01_eeg.set:
	[ EEG has 36 channels]
	[ 33 EEG channels, 2 EOG channels, 0 OTHER channels 1 MISC channels]
	[ Saved as sub-002_task-AuditoryVisualShift_run-01_channels.tsv]
	[ Adjusted the MiscChannelCount to 1]
sub-003_task-AuditoryVisualShift_run-01_eeg.set:
	[ EEG has 36 channels]
	[ 33 EEG channels, 2 EOG channels, 0 OTHER channels 1 MISC channels]
	[ Saved as sub-003_task-AuditoryVisualShift_run-01_channels.tsv]
	[ Adjusted the MiscChannelCount to 1]
sub-004_task-AuditoryVisualShift_run-01_eeg.set:
	[ EEG has 36 channels]
	[ 33 EEG channels, 2 EOG channels, 0 OTHER channels 1 MISC channels]
	[ Saved as sub-004_task-AuditoryVisualShift_run-01